In [1]:
import pandas as pd #dataframes

In [2]:
#create a dataframe for the months needed after importing the files
df = pd.DataFrame()
for month in ['03','04','05','06']:
    df = df.append(pd.read_csv("original_data/green_tripdata_2019-"+month+".csv" , low_memory = False), ignore_index=True)
    
#rename the location id to match the taxi zone location id          
df.rename(columns={'PULocationID':'LocationID'}, inplace=True)
df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'], format='%Y-%m-%d %H:%M:%S')

In [8]:
#create a new df from selected columns
df2 = df[['LocationID', 'lpep_pickup_datetime', 'passenger_count']]

#change the datetime to date
df2['lpep_pickup_datetime'] = df2['lpep_pickup_datetime'].dt.date
green_taxi_2019=df2[['lpep_pickup_datetime', 'passenger_count']]
green_taxi_2019

<ipython-input-8-88bc025f9c19>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['lpep_pickup_datetime'] = df2['lpep_pickup_datetime'].dt.date


,lpep_pickup_datetime,passenger_count
0,2019-03-01,1
1,2019-03-01,1
2,2019-03-01,1
3,2019-02-28,1
4,2019-03-01,1
...,...,...
2091428,2019-06-30,1
2091429,2019-06-30,1
2091430,2019-06-30,1
2091431,2019-06-30,1


In [9]:
#create a dataframe by merging with taxi zone
zone_lookup = pd.read_csv("original_data/taxi_zone_lookup.csv") # this dataset have a map between the LocationID and Borough
green_taxi_2019_boroughs = df2.merge(zone_lookup[['LocationID', 'Borough']], how='inner' ,on='LocationID').fillna(0)

#drop unnecessary column
green_taxi_2019_boroughs = green_taxi_2019_boroughs.drop(['LocationID'], axis=1)
green_taxi_2019_boroughs.head()

,lpep_pickup_datetime,passenger_count,Borough
0,2019-03-01,1,Queens
1,2019-03-01,1,Queens
2,2019-03-01,2,Queens
3,2019-03-01,1,Queens
4,2019-03-01,1,Queens


In [10]:
#create a dataframe with only usage
green_2019= pd.DataFrame(df2.lpep_pickup_datetime.value_counts().reset_index().values, columns=["Date", "Green_Taxi_2019"])
green_2019 = green_2019.sort_values(by="Date")
green_2019

,Date,Green_Taxi_2019
127,2008-12-31,6
123,2009-01-01,21
125,2010-09-23,15
130,2018-03-07,1
128,2019-02-14,5
...,...,...
85,2019-06-27,15797
43,2019-06-28,17505
82,2019-06-29,16151
119,2019-06-30,13183


In [11]:
#save the dataframe as csv file
green_2019.to_csv("green_2019.csv")

In [12]:
#create a dataframe to get the usage by borough
green_2019_borough = green_taxi_2019_boroughs.groupby("Borough").count().reset_index()
green_2019_borough.rename(columns={'lpep_pickup_datetime':'Green_Taxi_2019'}, inplace=True)
green_2019_borough=green_2019_borough.drop(['passenger_count'], axis=1)
green_2019_borough

,Borough,Green_Taxi_2019
0,Bronx,134417
1,Brooklyn,596974
2,EWR,25
3,Manhattan,742145
4,Queens,611147
5,Staten Island,967
6,Unknown,5758


In [13]:
#save the dataframe as csv file
green_2019_borough.to_csv("green_2019_borough.csv")